In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from accelerate import dispatch_model
from accelerate.utils import infer_auto_device_map
import accelerate

#model/thoth/thoth-1b2-v3
#model/Qwen2.5-Coder-3B-Instruct

model_name = "model/Qwen2.5-Coder-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    trust_remote_code=True
)

/opt/homebrew/Caskroom/miniconda/base/envs/myenv/lib/python3.8/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


FlashAttention2 is not installed.


InternLM2ForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)
prompt = "write a quick sort algorithm."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print(text)

<s><|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
write a quick sort algorithm.<|im_end|>
<|im_start|>assistant



In [7]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
print(model_inputs)

{'input_ids': tensor([[    1,     1, 92543,  9081,   364,  2770,   657,  1353, 17085,   328,
          3627,   684, 53374, 14947,   281,  1592,   657,   395, 11100, 17993,
           281, 92542,   364, 92543,  1008,   364,  5093,   395,  4131,  3541,
         12265,   281, 92542,   364, 92543,   525, 11353,   364]],
       device='mps:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='mps:0')}


In [9]:

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)


AssertionError: 

In [5]:
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Certainly! Quick Sort is a popular and efficient sorting algorithm that uses a divide-and-conquer approach. Below is a simple implementation of the Quick Sort algorithm in Python:

```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[len(arr) // 2]  # Choosing the middle element as the pivot
        left = [x for x in arr if x < pivot]  # Elements less than the pivot
        middle = [x for x in arr if x == pivot]  # Elements equal to the pivot
        right = [x for x in arr if x > pivot]  # Elements greater than the pivot
        return quick_sort(left) + middle + quick_sort(right)

# Example usage:
array = [3, 6, 8, 10, 1, 2, 1]
sorted_array = quick_sort(array)
print(sorted_array)
```

### Explanation:
1. **Base Case**: If the array has 0 or 1 element, it is already sorted, so we return it as is.
2. **Pivot Selection**: We choose the middle element of the array as the pivot.
3. **Partitioning**:
   - We create three lists: `left` for 